<a href="https://colab.research.google.com/github/RodrigoLima82/Tensorflow2.0/blob/master/Preparando_um_modelo_TensorFlow_para_um_dispositivo_m%C3%B3vel_com_TensorFlow_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Etapa 1: Instalação das bibliotecas

In [1]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 368kB 39.5MB/s 
     |████████████████████████████████| 3.2MB 25.6MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


## Etapa 2: Importação das bibliotecas

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

'1.13.1'

## Etapa 3: Pré-processamento

### Carregando a base de dados FashionMNIST

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Normalização das imagens

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

### Mudando a dimensionalidade da base de dados

In [5]:
X_train.shape

(60000, 28, 28)

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

## Etapa 4: Construindo o modelo

### Definindo o modelo

In [0]:
model = tf.keras.models.Sequential()

### Construindo o modelo

In [9]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compilando o modelo

In [10]:
y_test

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Treinando o modelo

In [12]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 9s 148us/sample - loss: 0.5263 - sparse_categorical_accuracy: 0.8139
Epoch 2/5
60000/60000 [==============================] - 8s 138us/sample - loss: 0.3984 - sparse_categorical_accuracy: 0.8570
Epoch 3/5
60000/60000 [==============================] - 8s 140us/sample - loss: 0.3658 - sparse_categorical_accuracy: 0.8671
Epoch 4/5
60000/60000 [==============================] - 8s 138us/sample - loss: 0.3443 - sparse_categorical_accuracy: 0.8731
Epoch 5/5
60000/60000 [==============================] - 8s 136us/sample - loss: 0.3271 - sparse_categorical_accuracy: 0.8802


### Avaliando o modelo

In [13]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

10000/10000 [==============================] - 1s 79us/sample - loss: 0.3510 - sparse_categorical_accuracy: 0.8732
Test accuracy: 0.873199999332428


## Etapa 5: Convertendo o modelo para o TensorFlow Lite

### Salvando o modelo

In [0]:
model_name = "fashion_mobile_model.h5"
tf.keras.models.save_model(model, model_name)

### Criando o TFLite Converter

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


### Convertendo o modelo

In [0]:
tflite_model = converter.convert()

### Salvando a versão TFLite

In [0]:
with open("tflite_model", "wb") as f:
  f.write(tflite_model)